# CRS Examples

This example demonstrates how one table with a EPSG 4326 CRS cannot be joined with another table that uses EPSG 3857.

In [1]:
import sedonadb

sd = sedonadb.connect()

Create a table with a geometry column that uses EPSG 4326.

In [2]:
countries = sd.read_parquet(
    "https://raw.githubusercontent.com/geoarrow/geoarrow-data/v0.2.0/natural-earth/files/natural-earth_countries_geo.parquet"
)

In [3]:
countries.schema

SedonaSchema with 3 fields:
  name: Utf8View
  continent: Utf8View
  geometry: wkb_view <epsg:4326>

In [6]:
cities = sd.sql("""
SELECT city, ST_SetSRID(ST_GeomFromText(wkt), 3857) AS geometry FROM (VALUES
    ('New York', 'POINT(-8238310.24 4969803.34)'),
    ('Los Angeles', 'POINT(-13153204.78 4037636.04)'),
    ('Chicago', 'POINT(-9757148.04 5138517.44)'))
AS t(city, wkt)""")

In [7]:
cities.schema

SedonaSchema with 2 fields:
  city: Utf8
  geometry: wkb <epsg:3857>

In [8]:
cities.to_view("cities", overwrite=True)
countries.to_view("countries", overwrite=True)

In [9]:
# join doesn't work when CRSs don't match
sd.sql("""
select * from cities
join countries
where ST_Intersects(cities.geometry, countries.geometry)
""").show()

SedonaError: type_coercion
caused by
Error during planning: Mismatched CRS arguments: epsg:3857 vs epsg:4326
Use ST_Transform() or ST_SetSRID() to ensure arguments are compatible.

In [17]:
# update cities to use 4326
cities = sd.sql("""
SELECT city, ST_Transform(ST_SetSRID(geometry, 3857), 4326) as geometry
FROM cities
""")

In [18]:
cities.schema

SedonaSchema with 2 fields:
  city: Utf8
  geometry: wkb

In [19]:
cities.to_view("cities", overwrite=True)

In [20]:
# join works when CRSs match
sd.sql("""
select * from cities
join countries
where ST_Intersects(ST_Transform(cities.geometry, 4326), countries.geometry)
""").show()

SedonaError: type_coercion
caused by
Error during planning: Mismatched CRS arguments: None vs epsg:4326
Use ST_Transform() or ST_SetSRID() to ensure arguments are compatible.